In [5]:
from pathlib import Path
import numpy as np
import os
import pickle

__file__ = Path(os.path.abspath('__file__'))
root_dir = __file__.parent
data_dir = root_dir/'wok'/'data'
pickle_dir = root_dir/'wok'/'data'/'pickle'

In [6]:
with open(pickle_dir/'atoms.pickle', 'rb') as f:
    atom_ids = pickle.load(f)

with open(pickle_dir/'nonatoms_decomposed.pickle', 'rb') as f:
    nonatom_ids = pickle.load(f)

print(f"atoms_ids: {len(atom_ids)}")
print(f"nonatoms_ids: {len(nonatom_ids)}")

ids = np.concatenate((atom_ids, nonatom_ids), axis=0)
print(f"total_ids: {len(ids)}")

atoms_ids: 469
nonatoms_ids: 88480
total_ids: 88949


In [7]:
nonatom_ids

array([['0x2e80', '⺀', '⿱丶丶'],
       ['0x2eb8', '⺸', '⿱艹干'],
       ['0x2ebb', '⺻', '⿱⿻⿻コ一丨一'],
       ...,
       ['0x2fa1b', '鼖', '⿱⿱十⿰𠂇十⿰⿱十豆⿱十又'],
       ['0x2fa1c', '鼻', '⿱自⿱田⿻一⿰丿丨'],
       ['0x2fa1d', '𪘀', '⿰⿱止⿶凵⿳⿰人人一⿰人人⿰⿱丿干⿱丿干']], dtype=object)